In [1]:
import nltk
nltk.download('punkt')
from nltk import *
import numpy as np
from urllib import request
from collections import Counter
import matplotlib.pyplot as plt
from sklearn import svm
from scipy.optimize import curve_fit
from PyDictionary import PyDictionary
from vocabulary.vocabulary import Vocabulary as vb

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Meet\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# import gzip
# url = "http://www.gutenberg.org/cache/epub/11/pg11.txt"  # Corpus used: The Adventures of Sherlock Holmes
# res = request.urlopen(url)
# gzipFile = gzip.GzipFile(fileobj=res)
# text = str(gzipFile.read())

In [3]:
url = "http://www.gutenberg.org/cache/epub/1661/pg1661.txt"  # Corpus used: The Adventures of Sherlock Holmes
res = request.urlopen(url)
text = res.read().decode('utf8')
# text= text.lower()
# loading the corpus from the Ebook itself hence NOTE that there might be some extra text in the corpus too

In [4]:
sent_tokenize_list = sent_tokenize(text)
word_tokenize_list = word_tokenize(text)
#tweet_tokenizer = TweetTokenizer()
#tweet_tokenizer_list = tweet_tokenizer.tokenize(text)

In [5]:
import string
translate_table_1 = dict((ord(char), None) for char in string.punctuation)   
translate_table_2 = dict((ord(char), " ") for char in string.whitespace[1:])

sent_tokenize_filtered_list_og = [(sent_tokenize_list[i].translate(translate_table_1)).translate(translate_table_2)
                          for i in range(2,len(sent_tokenize_list)-117)]
sent_tokenize_filtered_list_og = [" SsSs "+sent_tokenize_filtered_list_og[i]+" EeEe " for i in range(len(sent_tokenize_filtered_list_og))]
# HERE SsSs is thee tag/marker used forrepresenting the start of the sentence instead of <s>. Similarly EeEe is 
# used to represent the end of the sentence instead of <e> 

In [6]:
import sklearn
sent_tokenize_filtered_list, sent_tokenize_filtered_list_test = sklearn.model_selection.train_test_split(
    sent_tokenize_filtered_list_og, test_size = 0.2)
# used for parsing the entire dataset into training and test sets in the ratio of 80% to 20% respectively

In [7]:
len(sent_tokenize_filtered_list)

5464

In [8]:
mod_text = "".join(sent_tokenize_filtered_list)
# tokens = sorted(set(word_tokenize_list),key = word_tokenize_list.index)
tokens = word_tokenize(mod_text)
translate_table_3 = dict((ord(char), None) for char in string.punctuation[0:17] or string.punctuation[18] or string.punctuation[20:])
tokens = [tokens[i].translate(translate_table_3) for i in range(len(tokens))]
for i in range(len(tokens)):
    if tokens[i].isalpha()==False:
        tokens[i]=""
tokens = [value for value in tokens if value != ""]

# above code used for filtering out punctuations and numbers and only accepts alphabets

# for i in range(len(tokens)):
#    if tokens[i].isalpha()==False:
#        tokens[i]=""
# tokens = sorted(set(tokens),key = tokens.index)

In [9]:
from collections import Counter
unigrams = list(nltk.ngrams(tokens, 1))
bigrams = list(nltk.ngrams(tokens, 2))
trigrams = list(nltk.ngrams(tokens, 3))
quadgrams = list(nltk.ngrams(tokens, 4))

In [10]:
count_unigrams = Counter(unigrams)
count_bigrams = Counter(bigrams)
count_trigrams = Counter(trigrams)
count_quadgrams = Counter(quadgrams)
no_of_tokens = len(tokens)

In [11]:
# Here training corpus is used for computing the MLE
# MLE corresponding to the entire corpus has been computed in cell [32]
prob_unigrams = dict()
for i in range(len(unigrams)):
    prob_unigrams[unigrams[i]] = count_unigrams[unigrams[i]]/no_of_tokens 
    
prob_bigrams = dict()
for i in range(len(bigrams)):
    prob_bigrams[bigrams[i]] = count_bigrams[bigrams[i]]/count_unigrams[(bigrams[i][0],)] 

prob_trigrams = dict()
for i in range(len(trigrams)):
    prob_trigrams[trigrams[i]] = count_trigrams[trigrams[i]]/count_bigrams[(trigrams[i][0],trigrams[i][1])] 
    
prob_quadgrams = dict()
for i in range(len(quadgrams)):
    prob_quadgrams[quadgrams[i]] = count_quadgrams[quadgrams[i]]/count_trigrams[(quadgrams[i][0],quadgrams[i][1],quadgrams[i][2])] 

In [12]:
# Here tokens contain just the training data and not the entire corpus hence n-grams also correspond to the training data and 
# not the entire corpus

from math import factorial as fac

len_types = len(set(tokens)) 

# from itertools import permutations, combinations

# total_bigrams_ordered = len(list(permutations(list(set(tokens)),2)))
# total_bigrams_order_not_imp = len(list(combinations(list(set(tokens)),2)))

# total_trigrams_ordered = len(list(permutations(list(set(tokens)),3)))
# total_trigrams_order_not_imp = len(list(combinations(list(set(tokens)),3)))

# total_quadgrams_ordered = len(list(permutations(list(set(tokens)),4)))
# total_quadgrams_order_not_imp = len(list(combinations(list(set(tokens)),4)))

print("Total no of possible Bigrams Order Important: "+str(int(fac(len_types)/fac(len_types-2))))
print("Total Bigrams Order Not Important: "+str(int(fac(len_types)/(fac(len_types-2)*fac(2)))))

print("Total no of possible Trigrams Order Important: "+str(int(fac(len_types)/fac(len_types-3))))
print("Total Trigrams Order Not Important: "+str(int(fac(len_types)/(fac(len_types-3)*fac(3)))))

print("Total no of possible Quadgrams Order Important: "+str(int(fac(len_types)/fac(len_types-4))))
print("Total Quadgrams Order Not Important: "+str(int(fac(len_types)/(fac(len_types-4)*fac(4)))))

print("#Bigrams in the training set:"+str(len(set(bigrams))))
print("#Trigrams in the training set:"+str(len(set(trigrams))))
print("#Quadgrams in the training set:"+str(len(set(quadgrams))))

Total no of possible Bigrams Order Important: 64056012
Total Bigrams Order Not Important: 32028006
Total no of possible Trigrams Order Important: 512576208024
Total Trigrams Order Not Important: 85429368004
Total no of possible Quadgrams Order Important: 4101122240400024
Total Quadgrams Order Not Important: 170880093350001
#Bigrams in the training set:42847
#Trigrams in the training set:72205
#Quadgrams in the training set:86314


In [13]:
def Probability(sentence, model_name):  # in logspace
    
    sentence = ' SsSs '+sentence+' EeEe '
    words = word_tokenize(sentence)
    len_words = len(words)
    
    if model_name==unigrams:
        result_prob = 0
        for i in range(len(words)):
            result_prob = result_prob + prob_unigrams[(words[i],)]
    
    if model_name==bigrams:
        result_prob = 0 #prob_unigrams[(words[0],)]
        for i in range(1,len(words)):
            result_prob = result_prob + prob_bigrams[(words[i-1],words[i])]
    
    if model_name==trigrams:
        result_prob = prob_bigrams[(words[0],words[1])] #prob_unigrams[(words[0],)] + prob_bigrams[(words[0],words[1])]
        for i in range(2,len(words)):
            result_prob = result_prob + prob_trigrams[(words[i-2],words[i-1],words[i])]
            
    if model_name==quadgrams:
        result_prob = prob_bigrams[(words[0],words[1])] + prob_trigrams[(words[0],words[1],words[2])]#prob_unigrams[(words[0],)] + prob_bigrams[(words[0],words[1])] + prob_trigrams[(words[0],words[1],words[2])]
        for i in range(3,len(words)):
            result_prob = result_prob + prob_quadgrams[(words[i-3],words[i-2],words[i-1],words[i])]
    
    return result_prob
        

In [14]:
def Generator(model_name):
    
    import operator
    import random
    prob_unigrams_sorted = dict(sorted(prob_unigrams.items(), key=operator.itemgetter(1))[::-1])
    prob_bigrams_sorted = dict(sorted(prob_bigrams.items(), key=operator.itemgetter(1))[::-1])
    prob_trigrams_sorted = dict(sorted(prob_trigrams.items(), key=operator.itemgetter(1))[::-1])
    prob_quadgrams_sorted = dict(sorted(prob_quadgrams.items(), key=operator.itemgetter(1))[::-1])
    
    if model_name==bigrams:
        sentence=["SsSs"]
        word=sentence[0]
        counter = 0
        while word!="EeEe" and counter<=25:
            candidate_bigrams = []
            candidate_bigrams_keys = list(prob_bigrams_sorted.keys())
            for i in range(len(candidate_bigrams_keys)):
                if candidate_bigrams_keys[i][0]==word:
                    candidate_bigrams = candidate_bigrams + [candidate_bigrams_keys[i]]
            if word==sentence[0]:
                word = random.choice(candidate_bigrams)[1]
            else:
#                result = random.choice(candidate_bigrams)
                maximum = 0.0
                for i in range(len(candidate_bigrams)):
                    if prob_bigrams_sorted[candidate_bigrams[i]]>maximum:
                        result=candidate_bigrams[i]
                word = result[1]
            sentence = sentence + [word]
            counter = counter + 1
            #print(sentence)
        if counter>25:
            sentence = sorted(set(sentence),key = sentence.index)
        return " ".join(sentence)
    
    if model_name==trigrams:
        sentence=["SsSs"]
        word=sentence[0]
        candidate_bigrams = []
        candidate_bigrams_keys = list(prob_bigrams_sorted.keys())
        for i in range(len(candidate_bigrams_keys)):
            if candidate_bigrams_keys[i][0]==word:
                candidate_bigrams = candidate_bigrams + [candidate_bigrams_keys[i]]
        if word==sentence[0]:
            word = random.choice(candidate_bigrams)[1]
        sentence = sentence + [word]
        while word!="EeEe":
            candidate_trigrams = []
            candidate_trigrams_keys = list(prob_trigrams_sorted.keys())
            for i in range(len(candidate_trigrams_keys)):
                if candidate_trigrams_keys[i][0]==sentence[-2] and candidate_trigrams_keys[i][1]==sentence[-1]:
                    candidate_trigrams = candidate_trigrams + [candidate_trigrams_keys[i]]
#            result = random.choice(candidate_trigrams)
            maximum = 0.0
            for i in range(len(candidate_trigrams)):
                if prob_trigrams_sorted[candidate_trigrams[i]]>maximum:
                    result=candidate_trigrams[i]
            word = result[2]
            sentence = sentence + [word]
        return " ".join(sentence)
    
    if model_name==quadgrams:
        sentence=["SsSs"]
        word=sentence[0]
        candidate_trigrams = []
        candidate_trigrams_keys = list(prob_trigrams_sorted.keys())
        for i in range(len(candidate_trigrams_keys)):
            if candidate_trigrams_keys[i][0]==word:
                candidate_trigrams = candidate_trigrams + [candidate_trigrams_keys[i]]
        if word==sentence[0]:
            trigram = random.choice(candidate_trigrams)
        sentence = sentence + [trigram[1]] + [trigram[2]]
        while word!="EeEe":
            candidate_quadgrams = []
            candidate_quadgrams_keys = list(prob_quadgrams_sorted.keys())
            for i in range(len(candidate_quadgrams_keys)):
                if candidate_quadgrams_keys[i][0]==sentence[-3] and candidate_quadgrams_keys[i][1]==sentence[-2] and candidate_quadgrams_keys[i][2]==sentence[-1]:
                    candidate_quadgrams = candidate_quadgrams + [candidate_quadgrams_keys[i]]
#            result = random.choice(candidate_quadgrams) 
            maximum = 0.0
            for i in range(len(candidate_quadgrams)):
                if prob_quadgrams_sorted[candidate_quadgrams[i]]>maximum:
                    result=candidate_quadgrams[i]
            word = result[3]
            sentence = sentence + [word]
        return " ".join(sentence)
    
    if model_name==unigrams:
        sentence=["SsSs"]
        word=sentence[0]
        while word!="EeEe":
            candidate_unigrams_keys = list(prob_unigrams_sorted.keys())
            result = random.choice(candidate_unigrams_keys)
            word=result[0]
            sentence=sentence + [word]
        return " ".join(sentence)
        

In [15]:
Generator(bigrams)

'SsSs Within there glimmered little reputation such tricks with just treat myself into two hard black lines while away these signs that foul tongue over upon each'

In [16]:
Generator(trigrams)

'SsSs Because there are sentimental considerations in the history of the office after me EeEe'

In [18]:
Generator(quadgrams)

'SsSs The case is as plain as a pikestaff and the more ready to engage in an affair when there is some stiffness in the working of it and I want you to come tonight by the last train EeEe'

In [19]:
uniq_tokens = sorted(set(tokens),key = tokens.index)
voc_len = len(uniq_tokens)
count_bigrams_smoothed = dict() 
for (first,second) in prob_bigrams:
    count_bigrams_smoothed[(first,second)] = ((count_bigrams[(first,second)] + 1) * count_unigrams[(first,)])/(count_unigrams[(first,)] + voc_len)

In [20]:
count_bigrams_diff=dict()
for k, v in count_bigrams.items():
    count_bigrams_diff[k] = v - count_bigrams_smoothed.get(k, 0)

In [21]:
from collections import OrderedDict
OrderedDict(sorted(count_bigrams_diff.items(), key=lambda t: t[1], reverse=True))

OrderedDict([(('EeEe', 'SsSs'), 3246.2420552420554),
             (('SsSs', 'I'), 472.65518265518267),
             (('of', 'the'), 445.86755098000395),
             (('in', 'the'), 340.289050782087),
             (('SsSs', 'It'), 198.68399168399168),
             (('to', 'the'), 197.66128237959225),
             (('I', 'have'), 181.94416047834892),
             (('at', 'the'), 167.63275520893959),
             (('that', 'I'), 166.30410547754315),
             (('SsSs', 'The'), 158.86605286605288),
             (('upon', 'the'), 151.8939828080229),
             (('SsSs', 'He'), 141.63142263142265),
             (('with', 'a'), 133.4041690793283),
             (('it', 'EeEe'), 130.12596856320567),
             (('SsSs', 'You'), 129.15117315117317),
             (('It', 'is'), 128.53008870774394),
             (('and', 'I'), 126.43231099824185),
             (('to', 'be'), 119.61577858760958),
             (('it', 'is'), 117.72216072614567),
             (('I', 'was'), 113.2436107628508)

Above output cell gives an Ordered Dictionary which contains bigrams with maximum count difference after add 1 smoothing. From the top bigrams, it can be concluded that bigrams whise frequency is more in the training corpus appears to have more count difference after add 1 smoothing

In [22]:
count_bigrams_values = np.array(list(count_bigrams.values()))
good_turing_N = dict()
good_turing_count_smoothed = dict()
num = 11
for i in range(1,num+1):
    good_turing_N[i] = np.where(count_bigrams_values==i)[0].shape[0]

for i in range(1,num):
    good_turing_count_smoothed[i] = ((i + 1) * good_turing_N[i+1])/good_turing_N[i]
    

In [23]:
good_turing_count_diff=dict()
for k, v in good_turing_count_smoothed.items():
    good_turing_count_diff[k] = k - v

In [24]:
good_turing_count_smoothed

{1: 0.29913616800463966,
 2: 1.0946938775510204,
 3: 2.1297539149888145,
 4: 2.8256302521008405,
 5: 4.014869888475836,
 6: 5.347222222222222,
 7: 4.712727272727273,
 8: 7.333333333333333,
 9: 10.454545454545455,
 10: 8.289855072463768}

In [25]:
good_turing_N

{1: 32761,
 2: 4900,
 3: 1788,
 4: 952,
 5: 538,
 6: 360,
 7: 275,
 8: 162,
 9: 132,
 10: 138,
 11: 104}

In [26]:
good_turing_count_diff

{1: 0.7008638319953604,
 2: 0.9053061224489796,
 3: 0.8702460850111855,
 4: 1.1743697478991595,
 5: 0.985130111524164,
 6: 0.6527777777777777,
 7: 2.287272727272727,
 8: 0.666666666666667,
 9: -1.454545454545455,
 10: 1.7101449275362324}

In [27]:
# uniq_tokens = sorted(set(tokens),key = tokens.index)
# voc_len = len(uniq_tokens)
# count_bigrams_smoothed = dict() 
# for i in range(len(uniq_tokens)):
#     for j in range(len(uniq_tokens)):
#         if (uniq_tokens[i],uniq_tokens[j]) in prob_bigrams:
#             count_bigrams_smoothed[(uniq_tokens[i],uniq_tokens[j])] = ((count_bigrams[(uniq_tokens[i],uniq_tokens[j])] + 1) * count_unigrams[(uniq_tokens[i],)])/(count_unigrams[(uniq_tokens[i],)] + voc_len)
#         else:
#             count_bigrams_smoothed[(uniq_tokens[i],uniq_tokens[j])] = count_unigrams[(uniq_tokens[i],)]/(count_unigrams[(uniq_tokens[i],)] + voc_len)
             

In [28]:
# mod_text_og = "".join(sent_tokenize_filtered_list_og)
# # tokens = sorted(set(word_tokenize_list),key = word_tokenize_list.index)
# tokens_og = word_tokenize(mod_text_og)
# uniq_tokens = sorted(set(tokens_og),key = tokens_og.index)
# voc_len = len(uniq_tokens)
# prob_bigrams_add1_smoothed = dict() 
# for i in range(len(uniq_tokens)):
#     for j in range(len(uniq_tokens)):
#         if (uniq_tokens[i],uniq_tokens[j]) in prob_bigrams:
#             prob_bigrams_add1_smoothed[(uniq_tokens[i],uniq_tokens[j])] = (count_bigrams[(uniq_tokens[i],uniq_tokens[j])] + 1)/(count_unigrams[(uniq_tokens[i],)] + voc_len)
#         else:
#             prob_bigrams_add1_smoothed[(uniq_tokens[i],uniq_tokens[j])] = 1/(count_unigrams[(uniq_tokens[i],)] + voc_len)
            

In [29]:
mod_text_og = "".join(sent_tokenize_filtered_list_og)
# tokens = sorted(set(word_tokenize_list),key = word_tokenize_list.index)
tokens_og = word_tokenize(mod_text_og)
uniq_tokens = sorted(set(tokens_og),key = tokens_og.index)
voc_len = len(uniq_tokens)

In [30]:
#result_prob_add1_smoothed = 1
perplexity_add1 = []
for i in range(len(sent_tokenize_filtered_list_test)):
    words = word_tokenize(sent_tokenize_filtered_list_test[i])
    words = [words[k].translate(translate_table_3) for k in range(len(words))]
    for z in range(len(words)):
        if words[z].isalpha()==False:
            words[z]=""
    words = [value for value in words if value != ""]
    res_prob = 1
    for j in range(len(words)-1):
        if (words[j],words[j+1]) in prob_bigrams:
            int_prob = (count_bigrams[(words[j],words[j+1])] + 1)/(count_unigrams[(words[j],)] + voc_len)
        else:
            int_prob = 1/(count_unigrams[(words[j],)] + voc_len)
        res_prob = res_prob * int_prob
        print(res_prob,int_prob)
    perp = ((1.0/res_prob)**(1.0/len(words)))
#     result_prob_add1_smoothed = result_prob_add1_smoothed * probab
    perplexity_add1 = perplexity_add1 + [perp] 

In [31]:
import statistics as sts
perplexity_add1_mean = sts.mean(perplexity_add1)

In [32]:
perplexity_add1_mean

1169.1780768686785

In [33]:
count_bigrams_values = np.array(list(count_bigrams.values()))
good_turing_N = dict()
good_turing_count_smoothed = dict()
for i in set(count_bigrams_values):
    good_turing_N[i] = np.where(count_bigrams_values==i)[0].shape[0]

# # for i in range(1,11):
# #     good_turing_count_smoothed[i] = ((i + 1) * good_turing_N[i+1])/good_turing_N[i]

In [34]:
good_turing_N

{1: 32761,
 2: 4900,
 3: 1788,
 4: 952,
 5: 538,
 6: 360,
 7: 275,
 8: 162,
 9: 132,
 10: 138,
 11: 104,
 12: 73,
 13: 48,
 14: 55,
 15: 43,
 16: 41,
 17: 25,
 18: 38,
 19: 28,
 20: 21,
 21: 25,
 22: 17,
 23: 18,
 24: 18,
 25: 12,
 26: 18,
 27: 19,
 28: 20,
 29: 9,
 30: 6,
 31: 7,
 32: 14,
 33: 8,
 34: 3,
 35: 6,
 36: 6,
 37: 7,
 38: 5,
 39: 5,
 40: 2,
 41: 5,
 42: 4,
 43: 3,
 44: 4,
 45: 5,
 46: 3,
 47: 1,
 48: 3,
 49: 1,
 50: 3,
 563: 1,
 52: 4,
 53: 4,
 54: 2,
 55: 1,
 56: 4,
 57: 1,
 58: 2,
 60: 1,
 61: 1,
 62: 2,
 63: 2,
 64: 3,
 65: 1,
 66: 2,
 67: 1,
 70: 1,
 71: 3,
 72: 2,
 74: 2,
 76: 3,
 77: 3,
 78: 2,
 79: 2,
 83: 2,
 84: 1,
 85: 2,
 87: 2,
 88: 2,
 91: 1,
 92: 2,
 93: 1,
 96: 3,
 100: 3,
 102: 1,
 103: 2,
 105: 2,
 106: 1,
 110: 1,
 117: 1,
 118: 1,
 119: 1,
 120: 1,
 126: 2,
 128: 1,
 131: 1,
 133: 1,
 134: 1,
 137: 1,
 138: 2,
 139: 1,
 144: 1,
 147: 2,
 152: 1,
 158: 1,
 159: 1,
 162: 1,
 180: 1,
 194: 1,
 218: 1,
 236: 1,
 239: 1,
 251: 1,
 268: 1,
 796: 1,
 335: 1,
 54

In [35]:
d = 1
i = 10
while (d >= 1.0 or d < 0.0):
    d = sts.median(list(good_turing_count_diff.values())[:i])
    i = i-1

In [36]:
d

0.8877761037300825

In [37]:
mod_text_og = "".join(sent_tokenize_filtered_list_og)
# tokens = sorted(set(word_tokenize_list),key = word_tokenize_list.index)
tokens_og = word_tokenize(mod_text_og)
translate_table_3 = dict((ord(char), None) for char in string.punctuation[0:17] or string.punctuation[18] or string.punctuation[20:])
tokens_og = [tokens_og[i].translate(translate_table_3) for i in range(len(tokens_og))]
for i in range(len(tokens_og)):
    if tokens_og[i].isalpha()==False:
        tokens_og[i]=""
tokens_og = [value for value in tokens_og if value != ""]
# for i in range(len(tokens)):
#    if tokens[i].isalpha()==False:
#        tokens[i]=""
# tokens = sorted(set(tokens),key = tokens.index)

from collections import Counter
unigrams_og = list(nltk.ngrams(tokens_og, 1))
bigrams_og = list(nltk.ngrams(tokens_og, 2))
trigrams_og = list(nltk.ngrams(tokens_og, 3))
quadgrams_og = list(nltk.ngrams(tokens_og, 4))

count_unigrams_og = Counter(unigrams_og)
count_bigrams_og = Counter(bigrams_og)
count_trigrams_og = Counter(trigrams_og)
count_quadgrams_og = Counter(quadgrams_og)
no_of_tokens_og = len(tokens_og)

prob_unigrams_og = dict()
for i in range(len(unigrams_og)):
    prob_unigrams_og[unigrams_og[i]] = count_unigrams_og[unigrams_og[i]]/no_of_tokens_og 
    
prob_bigrams_og = dict()
for i in range(len(bigrams_og)):
    prob_bigrams_og[bigrams_og[i]] = count_bigrams_og[bigrams_og[i]]/count_unigrams_og[(bigrams_og[i][0],)] 

prob_trigrams_og = dict()
for i in range(len(trigrams_og)):
    prob_trigrams_og[trigrams_og[i]] = count_trigrams_og[trigrams_og[i]]/count_bigrams_og[(trigrams_og[i][0],trigrams_og[i][1])] 
    
prob_quadgrams_og = dict()
for i in range(len(quadgrams_og)):
    prob_quadgrams_og[quadgrams_og[i]] = count_quadgrams_og[quadgrams_og[i]]/count_trigrams_og[(quadgrams_og[i][0],quadgrams_og[i][1],quadgrams_og[i][2])] 



In [38]:
#result_prob_add1_smoothed = 1
perplexity_good_turing = []
for i in range(len(sent_tokenize_filtered_list_test)):
    words = word_tokenize(sent_tokenize_filtered_list_test[i])
    words = [words[k].translate(translate_table_3) for k in range(len(words))]
    for z in range(len(words)):
        if words[z].isalpha()==False:
            words[z]=""
    words = [value for value in words if value != ""]
    res_prob = 1
    for j in range(len(words)-1):
        if (words[j],words[j+1]) in prob_bigrams:
            int_prob = (count_bigrams[(words[j],words[j+1])] - d)/(count_unigrams[(words[j],)])
        else:
            int_prob = (d/count_unigrams_og[(words[j],)])*prob_unigrams_og[(words[j+1],)]
        res_prob = res_prob * int_prob
        print(res_prob,int_prob)
    perp = ((1.0/res_prob)**(1.0/len(words)))
#     result_prob_add1_smoothed = result_prob_add1_smoothed * probab
    perplexity_good_turing = perplexity_good_turing + [perp] 

In [39]:
import statistics as sts
perplexity_good_turing_mean = sts.mean(perplexity_good_turing)

In [40]:
perplexity_good_turing_mean

1052.5785510999842